In [1]:
import time
import xlrd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import datetime
from datetime import datetime
from datetime import timedelta
import dateutil.relativedelta
import calendar
import re
import pandas
import pandas as pd

前面都是def funciton，最后把所有function汇集起来




我手头的项目是要爬2010年到现在每个月/每一天的关于同居的微博。目前常见的微博爬虫有三个目标网站，一个是pc端口s.weibo.com，一个是智能手机端口m.weibo.cn，一个是非常老的手机端口weibo.cn。很多微博爬虫都采用智能手机端端口，因为可以爬到很多数据，只要不停往下滑应该就能爬完。但是我目前还没有发现智能手机端口如何搜索指定的日期或时间段，这就不得不用pc端口或者非常老的那个手机端口了。

非常老的手机端口的好处在于不用管验证码，直接就可以登陆，省去了非常多的麻烦。而且不存在‘展开全文’的问题，pc端和智能手机端需要展开原文。而且每一个日期可以搜索到100页，pc端只能搜索到50页。虽然可能pc端每一页比手机端口要多，但是应该还是这个端口能爬到的更多一点。还有一个原因是可以直接勾选原创微博，省去了看上去爬了很多，但实际上爬完了还需要删除所有的转发的过程（虽然复制粘贴的还是要额外删除）。

用selenium是因为微博会反爬虫，selenium可以模拟正常登陆的流程，而且一定要在每一次点击一个页面之后停顿几秒钟（sleep），不然很容易被封号甚至封IP。

In [2]:
#我本来想按照月来爬的，但是输入指定月份例如20100101-20100131之后
#爬到的几乎都是1月31号的微博，这样的样本不是很好
#所以还是每天都爬2、3页当做样本，过几个星期我再每天都爬100页试试

def dates_in_year(YEAR):
    YEAR = int(YEAR)
    MONTH = 1
    DAY = 1
    date = datetime(YEAR,MONTH,DAY)
    
    days_in_year = 365 + (1*calendar.isleap(YEAR))
    for i in range(days_in_year):
        days_normal.append(date.strftime("%Y")+date.strftime("%m")+date.strftime("%d"))
        days_datetime.append(date) 
        date = date + dateutil.relativedelta.relativedelta(days=1)     

In [3]:
def login():   
    #登陆 Login
    driver.get('https://weibo.cn/pub/?vt=' )
   
    elem = driver.find_element_by_xpath("//*[@class='ut']/a[1]").click();
    time.sleep(1)
    
    print("开始自动登陆Loading")
    elem = driver.find_element_by_xpath("//*[@id='loginName']");
    elem.send_keys(username)
    elem = driver.find_element_by_xpath("//*[@id='loginPassword']");
    elem.send_keys(password)
    elem = driver.find_element_by_xpath("//*[@id='loginAction']");
    elem.send_keys(Keys.ENTER)  
    time.sleep(1)
    print('登陆完毕 Login Finish')

In [4]:
#每爬完一天清理一次临时数据
#clear temp data after each day
def clean_temp_data():
    usernames.clear()
    user_links.clear()
    posts.clear()
    likes.clear()
    reblogs.clear()
    comments.clear()
    comment_links.clear()
    post_date.clear()
    post_time.clear()

In [13]:
#每天保存
#save each day
def save_data():
    for post in posts:
        data['Posts'].append(post)
    for username in usernames:
        data['User Names'].append(username)
    for user_link in user_links:
        data['User Links'].append(user_link)
    for like in likes:
        data['Likes'].append(like)
    for reblog in reblogs:
        data['Reblogs'].append(reblog)
    for comment in comments:
        data['Comments'].append(comment)
    for comment_link in comment_links:
        data['Comment Links'].append(comment_link)
    for post_d in post_date:
        data['Post Date'].append(post_d)
    for post_t in post_time:
        data['Post Time'].append(post_t)
    for post_date_d in post_date_datetime:
        data['Date in Datetime Format'].append(post_date_d)

In [6]:
#翻页 Next Page
def click_next_page():
    NextPage = driver.find_element_by_xpath("//*[@class='pa']/form/div/a[1]")
            #driver.find_element_by_xpath("//a[contains(text(),'下页')]").click()
    NextPage.click()
    time.sleep(3)

In [7]:
#开始爬一页微博。注意elements是可以抓取多个，element只能抓一个
#Scraping one page. Notice elements not element. Elements can find multiple element

def scrape_one_page():
    #找到发微博的用户名和连接
    #Get username and href link
    elem = driver.find_elements_by_xpath("//*[@class='nk']")
    for post in elem:
        usernames.append(post.text)
        user_links.append(post.get_attribute('href'))

    #找到微博内容
    #Get post content
    elem = driver.find_elements_by_xpath("//*[@class='ctt']")
    for post in elem:
        posts.append(post.text)

    #点赞
    #likes
    likes_temp = []
    elem = driver.find_elements_by_xpath("//a[contains(text(),'赞')]")
    for like in elem:
        likes_temp.append(like.text)
    for like in likes_temp:
        like = re.sub(r'\D', '', like) #去除掉‘赞’这个字只保留数字Delete 'likes' text
        likes.append(like)
    
    #转发评论
    #reblogs
    reblog_temp = []
    elem = driver.find_elements_by_xpath("//a[contains(text(),'转发')]")
    for reblog in elem:
        reblog_temp.append(reblog.text)
    for reblog in reblog_temp:
        reblog = re.sub(r'\D', '', reblog) #保留数字keep the number
        reblogs.append(reblog)  

    #评论和评论的链接comment and links to comment section
    comment_temp = []
    elem = driver.find_elements_by_xpath("//a[contains(text(),'评论')]")
    for comment in elem:
        comment_temp.append(comment.text)
        comment_links.append(comment.get_attribute('href'))#评论的链接links to comment
    for comment in comment_temp:
        comment = re.sub(r'\D', '', comment) #保留数字keep the number
        comments.append(comment)  

    #发布日期和时间
    elem = driver.find_elements_by_xpath("//*[@class='ct']")
    for time in elem:
        post_date.append(time.text.split(' ')[0])
        post_time.append(time.text.split(' ')[1])

In [8]:
#爬一天内的每一页
#Scrape every page in a day
def scrape_everything_in_a_day(target_page_per_day):    
    
    current_page = []
    max_page = []
    
    #当前页数和最大页数
    #get current page and max page
    try:
        elem = driver.find_element_by_xpath("//*[@class='pa']")
    except NoSuchElementException:
            driver.back()
            time.sleep(5)
            driver.find_element_by_xpath("//*[@type='submit']").click()
            elem = driver.find_element_by_xpath("//*[@class='pa']")
    elem = elem.text
    elem = elem.split('/')
    current_page = elem[0]
    max_page = elem[1]
    current_page = re.sub(r'\D','',current_page)
    max_page = re.sub(r'\D','',max_page)
 
    #如果超过最大页码，目标页码变为最大页码
    if int(max_page) <target_page_per_day:
        target_page = max_page

    #开始循环, 如果被限制的话就返回停五秒钟再前进。   
    for page in range(int(target_page)):
        try:
            scrape_one_page()
            time.sleep(1)
        except NoSuchElementException:
            driver.back()
            time.sleep(5)
            try:
                click_next_page() #第一种情况点击下一页
            except NoSuchElementException:
                driver.find_element_by_xpath("//*[@type='submit']").click()
                #第二种情况点击搜索
            
            
        #有一些页面是空的所以用try except跳过去防止报错
        try:
            elem = driver.find_element_by_xpath("//*[@class='pa']")
            elem = elem.text
            elem = elem.split('/')
            current_page = elem[0]
            current_page = re.sub(r'\D','',current_page)
            #到最大页数的时候停止
            #stop when hitting max page
            if int(current_page) == int(max_page):
                #保存数据
                save_data()
                break
            else:
                click_next_page()
    
        except NoSuchElementException:
            driver.back()
            time.sleep(5)
            driver.find_element_by_xpath("//*[@type='submit']").click()
           
            elem = driver.find_element_by_xpath("//*[@class='pa']")
            elem = elem.text
            elem = elem.split('/')
            current_page = elem[0]
            current_page = re.sub(r'\D','',current_page)
        
            if int(current_page) == int(max_page)-1:
                #保存数据
                save_data()
                break
            else:
                try:
                    elem = driver.find_element_by_xpath("//*[@input='page']")
                except NoSuchElementException:
                    elem = driver.find_element_by_xpath("//*[@name='page']")
                jump_one_page = int(current_page) + 2
                elem.send_keys(jump_one_page)
                driver.find_element_by_xpath("//*[@value='跳页']").click()

接下来是搜索前后和按照日期循环

Below are before and after search and reiterate through dates

In [9]:
def open_search_page_first_time():
    #点开搜索页面 Open search tap
    driver.find_element_by_xpath("//a[contains(text(),'搜索')]").click()
    driver.find_element_by_xpath("//a[contains(text(),'高级搜索')]").click()
    time.sleep(1)
    
def keyword_input(key):
    #输入关键词 Input Keywords
    #我这里是"同居" In this case 'cohabitation'
    elem = driver.find_element_by_xpath("//*[@type='text']")
    elem.send_keys(key)
    
def date_input(day):
    #开始年月日 Start data e.g, 20200101
    elem = driver.find_element_by_xpath("//*[@name='starttime']")
    elem.clear()
    elem.send_keys(day)

    #结束年月日一样 End data same
    elem = driver.find_element_by_xpath("//*[@name='endtime']")
    elem.clear()
    elem.send_keys(day)

In [10]:
def search_first_time(key):
    #上面def的两个function
    open_search_page_first_time()
    keyword_input(key)
    
    #勾选原创 Click original posts
    driver.find_element_by_xpath("//*[@name='hasori']").click()
    
    date_input(days_normal[0])
    
    #搜索 Search
    driver.find_element_by_xpath("//*[@type='submit']").click()
    time.sleep(1)
    
def search_reiterate(day):
    #点开‘更多’ Open'more'
    #因为之前输入过关键词和勾选原创，这里可以省略
    #Because we have input keywords and click original, we can skip here
    driver.find_element_by_xpath("//a[contains(text(),'更多')]").click()
    time.sleep(2)
    
    date_input(day)
    
    #搜索 Search
    driver.find_element_by_xpath("//*[@type='submit']").click()
    time.sleep(1)

In [11]:
def scrape_by_date(key, target_year, target_page_per_day):
    
    #创建指定年份的日期列表
    #Create a list of dyas in target year
    dates_in_year(target_year)
    
    #登陆
    #login
    login()
    
    #开始搜索指定关键词的第一天的原创微博
    #search for keywords, click original, on the first day in the list
    search_first_time(key)
    
    #爬+保存，清除临时数据列表
    scrape_everything_in_a_day(target_page_per_day)
    for post in range(len(posts)):
        post_date_datetime.append(days_datetime[0])
    clean_temp_data()
    
    #从第二天开始循环，所以去掉列表里第一天
    #Reit from second day, so delete first day in the list
    days_normal_from_2nd_day = []
    days_normal_from_2nd_day = days_normal[1:]
    for day in days_normal_from_2nd_day:
        search_reiterate(day)
        scrape_everything_in_a_day(target_page_per_day)
        for post in range(len(posts)):
            post_date_datetime.append(datetime(int(day[:4]),
                                             int(day[4:6]),
                                             int(day[6:8])))
        print('Finish ' + day)
        clean_temp_data()
    
    print('Finish scraping ' + key + 'in ' + target_year + 'for '
         + target_page_per_day)

In [12]:
#创建临时数据列表
days_normal = []
days_datetime = []
usernames = []
user_links = []
posts = []
likes = []
reblogs = []
comments = []
comment_links = []
post_date = []   
post_time = []
post_date_datetime = []
data = {'Posts':[],
    'User Names':[],
    'User Links':[],
    'Likes':[],
    'Reblogs':[],
    'Comments':[],
    'Comment Links':[],
    'Post Date':[],
    'Post Time':[],
    'Date in Datetime Format':[]}


#正式跑起来
if __name__ == '__main__':
    username = "17064888409" #用户名username
    password = "rw297421" #密码password
    driver = webdriver.Chrome()#可以换成firefox.Can use firefox        
    scrape_by_date('同居', 2010,  5)
#页数可以设置成max_page

开始自动登陆Loading
登陆完毕 Login Finish


NameError: name 'post_date_datetimg' is not defined

In [ ]:
len(data['Posts'])

In [ ]:
len(data['User Names'])

In [ ]:
data['User Names']

In [ ]:
data['User Names']